In [1]:
%mkdir -p /content/drive/MyDrive/GSE146194
%cd /content/drive/MyDrive/GSE146194

/content/drive/MyDrive/GSE146194


In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE146194&format=file" -O GSE146194_RAW.tar
!tar -tf GSE146194_RAW.tar

--2024-08-03 14:16:51--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE146194&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6699868160 (6.2G) [application/x-tar]
Saving to: ‘GSE146194_RAW.tar’

GSE146194_RAW.tar   100%[===================>]   6.24G  8.62MB/s    in 14m 30s 

2024-08-03 14:31:22 (7.34 MB/s) - ‘GSE146194_RAW.tar’ saved [6699868160/6699868160]

GSM4367979_exp1-5.barcodes.tsv.gz
GSM4367979_exp1-5.cell_identities.csv.gz
GSM4367979_exp1-5.features.tsv.gz
GSM4367979_exp1-5.matrix.mtx.gz
GSM4367984_exp6.barcodes.tsv.gz
GSM4367984_exp6.cell_identities.csv.gz
GSM4367984_exp6.features.tsv.gz
GSM4367984_exp6.matrix.mtx.gz
GSM4367985_exp7.barcodes.tsv.gz
GSM4367985_exp7.cell_identities.csv.gz
GSM4367985_exp7.features.tsv.gz
GSM4367985_exp7.matrix.mtx.gz
GSM4367986_exp8.barco

In [ ]:
!tar -xvf GSE146194_RAW.tar

GSM4367979_exp1-5.barcodes.tsv.gz
GSM4367979_exp1-5.cell_identities.csv.gz
GSM4367979_exp1-5.features.tsv.gz
GSM4367979_exp1-5.matrix.mtx.gz
GSM4367984_exp6.barcodes.tsv.gz
GSM4367984_exp6.cell_identities.csv.gz
GSM4367984_exp6.features.tsv.gz
GSM4367984_exp6.matrix.mtx.gz
GSM4367985_exp7.barcodes.tsv.gz
GSM4367985_exp7.cell_identities.csv.gz
GSM4367985_exp7.features.tsv.gz
GSM4367985_exp7.matrix.mtx.gz
GSM4367986_exp8.barcodes.tsv.gz
GSM4367986_exp8.cell_identities.csv.gz
GSM4367986_exp8.features.tsv.gz
GSM4367986_exp8.matrix.mtx.gz
GSM4367987_exp9.barcodes.tsv.gz
GSM4367987_exp9.features.tsv.gz
GSM4367987_exp9.matrix.mtx.gz
GSM4367987_exp9.raw_cell_identities.csv.gz
GSM4367988_exp10.barcodes.tsv.gz
GSM4367988_exp10.features.tsv.gz
GSM4367988_exp10.matrix.mtx.gz
GSM4367988_exp10.raw_cell_identities.csv.gz
GSM4367989_exp11.barcodes.tsv.gz
GSM4367989_exp11.features.tsv.gz
GSM4367989_exp11.matrix.mtx.gz


In [2]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.8 MB/s eta 0:00:00


In [ ]:
samples = [file.split('.')[0] for file in os.listdir('.') if file.endswith('.mtx.gz')]
samples

['GSM4367979_exp1-5',
 'GSM4367984_exp6',
 'GSM4367986_exp8',
 'GSM4367987_exp9',
 'GSM4367988_exp10',
 'GSM4367989_exp11',
 'GSM4367985_exp7']

In [ ]:
cell_identities = {
    sample: file for sample in samples for file in os.listdir('.')
    if file.startswith(sample) and file.endswith('.csv.gz')
}
cell_identities

{'GSM4367979_exp1-5': 'GSM4367979_exp1-5.cell_identities.csv.gz',
 'GSM4367984_exp6': 'GSM4367984_exp6.cell_identities.csv.gz',
 'GSM4367986_exp8': 'GSM4367986_exp8.cell_identities.csv.gz',
 'GSM4367987_exp9': 'GSM4367987_exp9.raw_cell_identities.csv.gz',
 'GSM4367988_exp10': 'GSM4367988_exp10.raw_cell_identities.csv.gz',
 'GSM4367985_exp7': 'GSM4367985_exp7.cell_identities.csv.gz'}

In [ ]:
adatas = {}

for sample in cell_identities.keys():
    print(f'Processing {sample}')
    adata = sc.read_10x_mtx('.', prefix=f"{sample}.")
    cell_identity = pd.read_csv(cell_identities[sample], index_col=0)
    barcodes = np.intersect1d(adata.obs_names, cell_identity.index)
    adata = adata[barcodes]
    adata.obs = cell_identity.loc[barcodes]
    adatas[sample] = adata

adata = ad.concat(adatas, label='batch')
adata.write_h5ad('GSE146194_merged.h5ad')
adata

Processing GSM4367979_exp1-5
Only considering the two last: ['.mtx', '.gz'].
Only considering the two last: ['.mtx', '.gz'].
Processing GSM4367984_exp6
Only considering the two last: ['.mtx', '.gz'].
Only considering the two last: ['.mtx', '.gz'].
Processing GSM4367986_exp8
Only considering the two last: ['.mtx', '.gz'].
Only considering the two last: ['.mtx', '.gz'].
Processing GSM4367987_exp9
Only considering the two last: ['.mtx', '.gz'].
Only considering the two last: ['.mtx', '.gz'].
Processing GSM4367988_exp10
Only considering the two last: ['.mtx', '.gz'].
Only considering the two last: ['.mtx', '.gz'].
Processing GSM4367985_exp7
Only considering the two last: ['.mtx', '.gz'].
Only considering the two last: ['.mtx', '.gz'].


/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 259673 × 33694
    obs: 'guide_identity', 'read_count', 'UMI_count', 'coverage', 'gemgroup', 'good_coverage', 'number_of_cells', 'batch'

In [ ]:
adata.obs['guide_identity'].value_counts().tail()

,count
guide_identity,
204*NegCtrl4_posA|170*YBX1_+_43148542.23-P1P2_posB,1
204*NegCtrl4_posA|172*ATR_-_142297649.23-P1P2_posB,1
204*NegCtrl4_posA|172*RPS25_+_118888990.23-P1P2_posB,1
204*NegCtrl4_posA|172*SGTA_+_2783252.23-P1P2_posB,1
204*NegCtrl4_posA|137*SEC61A1_+_127783820.23-P2_posB,1


In [ ]:
adata.obs['guide_identity'].str.split('|').str.len().value_counts()

,count
guide_identity,
2,225270
1,34403


In [ ]:
adata = sc.read_h5ad('GSE146194_merged.h5ad')

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
import re

def assign_genes(row):
    if '|' in row:
        pattern1 = r'(\w+)_.*\|.*\*(\w+)_'
        pattern2 = r'sg(\w+)\|.*sg(\w+)'
        match1 = re.search(pattern1, row)
        if match1:
            return '+'.join(list(set([
                'non-targeting' if 'Ctrl' in gene else gene
                for gene in [match1.group(1), match1.group(2)]
            ])))
        match2 = re.search(pattern2, row)
        if match2:
            return '+'.join([sgrna.lstrip('sg') for sgrna in [match2.group(1), match2.group(2)]])
    return row.lstrip('sg') if 'Ctrl' not in row else 'non-targeting'

In [5]:
adata.obs['perturbation_name'] = adata.obs['guide_identity'].apply(assign_genes)
adata.obs['perturbation_name'].head()

,perturbation_name
cell_barcode,
AAACCCAAGAAACTCA-5,DDRGK1
AAACCCAAGAGGGCGA-4,TMEM167A
AAACCCAAGGAACATT-5,IER3IP1
AAACCCACAATCGCCG-5,SRP68
AAACCCACACCCTCTA-4,non-targeting


In [10]:
adata.obs['perturbation_name'].value_counts()

,count
perturbation_name,
non-targeting,19253
FDPSb+NegCtrl02093a,2575
HMGCRa+HUS1b,2095
non-targeting+HSF1,2025
HUS1b+NegCtrl02093a,2024
MTOR+non-targeting,2024
CREB1+non-targeting,2020
NegCtrl02093a+NegCtrl3b,1927
FDPSb+HUS1a,1907


In [6]:
adata.obs['condition'] = adata.obs['perturbation_name'].apply(
    lambda x: 'non-targeting' if x == 'non-targeting' else 'targeting'
)
adata.obs['condition'].value_counts()

,count
condition,
targeting,240420
non-targeting,19253


In [ ]:
adata.obs['batch'].value_counts()

,count
batch,
GSM4367985_exp7,114648
GSM4367986_exp8,60725
GSM4367984_exp6,37238
GSM4367979_exp1-5,34403
GSM4367987_exp9,6349
GSM4367988_exp10,6310


In [7]:
adata.obs['crispr_type'] = 'CRISPRi'
adata.obs.loc[adata.obs['batch'] == 'GSM4367986_exp8', 'crispr_type'] = 'CRISPRa'
adata.obs['cancer_type'] = 'Leukemia'
adata.obs['cell_type'] = 'leukemia cell'
adata.obs['organism'] = ORGANISMS[0]

In [8]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [9]:
adata.write_h5ad('GSE146194_merged.h5ad')

In [3]:
adata = sc.read_h5ad('GSE146194_merged.h5ad')

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
